In [21]:
!pip install "ibm-watsonx-ai" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install "langchain==0.1.8" | tail -n 1
!pip install "langchain_ibm==0.0.1" | tail -n 1

In [22]:
import getpass

credentials = {
    "url": "https://jp-tok.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ········


In [23]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [26]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}



In [27]:
from langchain_ibm import WatsonxLLM

granite_jp_llm = WatsonxLLM(
    model_id="ibm/granite-8b-japanese",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    params=parameters
    )
flan_t5_llm = WatsonxLLM(
    model_id=model_id_2,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
    )

In [28]:
granite_jp_llm.dict()


{'model_id': 'ibm/granite-8b-japanese',
 'deployment_id': '',
 'params': {'decoding_method': 'sample',
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '53afd616-fa11-4699-bd65-ea1e8ded9173',
 'space_id': '',
 '_type': 'IBM watsonx.ai'}

In [37]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"],
    template="""{topic}についてランダムな質問を生成してください。
                            質問: """
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="""次の質問に答えてください。: {question}
                            回答:""",
)

In [38]:
from langchain.chains import LLMChain

prompt_to_granite_jp = LLMChain(llm=granite_jp_llm, prompt=prompt_1, output_key='question')

In [39]:
granite_jp_2 = LLMChain(llm=granite_jp_llm, prompt=prompt_2, output_key='answer')

In [40]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_granite_jp, granite_jp_2], verbose=True)

In [41]:
qa.run("IBM")



> Entering new SimpleSequentialChain chain...
IBMの創立者は誰ですか？
 チャールズ・ライリー・ワトソン

> Finished chain.


' チャールズ・ライリー・ワトソン'